In [1]:
!wget -nc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

!conda create -y -n py38 python=3.8

!source /usr/local/bin/activate py38 && python --version

!source /usr/local/bin/activate py38 && conda install -y pip
!source /usr/local/bin/activate py38 && pip install torch==2.0.1 transformers==4.36.2 numpy==1.24.4

!git clone https://github.com/deepseek-ai/DeepSeek-VL.git
!source /usr/local/bin/activate py38 && pip install -e ./DeepSeek-VL

--2025-05-11 05:40:26--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155472915 (148M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 148.27M   216MB/s    in 0.7s    

2025-05-11 05:40:27 (216 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [155472915/155472915]

PREFIX=/usr/local
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the f

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Cytosine.png to Cytosine.png


In [6]:
!ls -lh /content

total 149M
-rw-r--r-- 1 root root  14K May 11 05:45 Acetic-acid.png
-rw-r--r-- 1 root root  26K May 11 05:45 Adenine.png
-rw-r--r-- 1 root root  12K May 11 05:45 Ammonia.png
-rw-r--r-- 1 root root  30K May 11 05:45 Benzoic_acid.png
-rw-r--r-- 1 root root  26K May 11 05:45 Caffeine.png
-rw-r--r-- 1 root root  18K May 11 05:51 Cytosine.png
drwxr-xr-x 7 root root 4.0K May 11 05:42 DeepSeek-VL
-rw-r--r-- 1 root root  17K May 11 05:45 Formic_acid.png
-rw-r--r-- 1 root root  21K May 11 05:45 H2O.png
-rw-r--r-- 1 root root  15K May 11 05:45 Histamine.png
-rw-r--r-- 1 root root  17K May 11 05:45 Ibuprofen.png
-rw-r--r-- 1 root root  20K May 11 05:45 Imidazole_full.png
-rw-r--r-- 1 root root  12K May 11 05:45 MeNH2.png
-rw-r--r-- 1 root root  15K May 11 05:45 Methanol.png
-rw-r--r-- 1 root root 149M Apr 30 20:14 Miniconda3-latest-Linux-x86_64.sh
-rw-r--r-- 1 root root  25K May 11 05:45 Morphine.png
-rw-r--r-- 1 root root  19K May 11 05:45 Nicotinamid.png
drwxr-xr-x 2 root root 4.0K May 11 05:47

In [3]:
with open("task2_deepseek.py", "w", encoding="utf-8") as f:
    f.write('''
from transformers import AutoTokenizer
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
import torch
from PIL import Image
import os
import csv

# --- Step 1: Load DeepSeek Model and Processor ---
model_id = "deepseek-ai/deepseek-vl-7b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiModalityCausalLM.from_pretrained(model_id, trust_remote_code=True).to(torch.bfloat16).cuda().eval()
vl_chat_processor = VLChatProcessor.from_pretrained(model_id)
tokenizer = vl_chat_processor.tokenizer

# --- Step 2: Set Dataset Folder and Image Pairs ---
dataset_folder = "/content"

image_pairs = [
    ("Benzoic_acid.png", "Phenol.png"),
    ("Formic_acid.png", "Acetic-acid.png"),
    ("Ammonia.png", "MeNH2.png"),
    ("Cytosine.png", "Adenine.png"),
    ("H2O.png", "Methanol.png"),
    ("Caffeine.png", "Morphine.png"),
    ("Ibuprofen.png", "Salicylic-acid.png"),
    ("Imidazole_full.png", "Pyridine-full.png"),
    ("Nicotinamid.png", "Histamine.png"),
    ("Purine.png", "Uracil.png"),
]

cot_types = ["Baseline", "Stepwise", "Visual_first", "Explanation_first"]

prompts = {
    "Baseline": "Which molecule is a stronger acid or base, the first or the second? Why?",
    "Stepwise": "Step 1: Identify the most acidic or basic site in each molecule. Step 2: Consider how stable the molecule is after gaining or losing a proton. Step 3: Say which molecule is stronger, the first or the second, and explain why.",
    "Visual_first": "Examine the image only. Based on visible atom types and bonds, which molecule appears stronger in acid/base behavior? End by stating 'first' or 'second'.",
    "Explanation_first": "The ability to stabilize charge after proton gain or loss determines acid/base strength. Analyze both molecules, then conclude which is stronger: first or second."
}

# --- Step 3: Output Setup ---
os.makedirs("outputs", exist_ok=True)
csv_file = open("outputs/Task2_deepseek_outputs.csv", "w", encoding="utf-8", newline="")
writer = csv.DictWriter(csv_file, fieldnames=["image1", "image2", "prompt_type", "prompt_text", "generated_text"])
writer.writeheader()

# --- Step 4: Run Inference ---
for img1, img2 in image_pairs:
    img_path1 = os.path.join(dataset_folder, img1)
    img_path2 = os.path.join(dataset_folder, img2)
    for cot in cot_types:
        prompt = prompts[cot]
        conversation = [
            {
                "role": "User",
                "content": prompt,
                "images": [img_path1, img_path2]
            },
            {
                "role": "Assistant",
                "content": ""
            }
        ]

        images = load_pil_images(conversation)
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=images,
            force_batchify=True
        ).to(model.device)

        with torch.no_grad():
            inputs_embeds = model.prepare_inputs_embeds(**prepare_inputs)
            outputs = model.language_model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=prepare_inputs.attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=512,
                do_sample=False,
                use_cache=True
            )

        answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
        print(f"--- Finished {cot} for {img1} and {img2}")
        writer.writerow({
            "image1": img1,
            "image2": img2,
            "prompt_type": cot,
            "prompt_text": prompt,
            "generated_text": answer.strip()
        })

csv_file.close()
print("!!!! All results saved to outputs/Task2_deepseek_outputs.csv")
''')


In [7]:
!source /usr/local/bin/activate py38 && python task2_deepseek.py

/usr/local/envs/py38/lib/python3.8/site-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.51s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can 

In [8]:
from google.colab import files
files.download('/content/outputs/Task2_deepseek_outputs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
with open("task2-deepseek-finetune.py", "w", encoding="utf-8") as f:
    f.write("""
from transformers import AutoTokenizer
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
import torch
from PIL import Image
import os
import csv

# ----- 1. Setup Model -----
model_id = "deepseek-ai/deepseek-vl-7b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiModalityCausalLM.from_pretrained(model_id, trust_remote_code=True).to(torch.bfloat16).cuda().eval()
vl_chat_processor = VLChatProcessor.from_pretrained(model_id)
tokenizer = vl_chat_processor.tokenizer

# ----- 2. Define Prompts (Baseline + 15 CoT prompts) -----
baseline_prompt = ["Which molecule is a stronger acid or base, the first or the second? Why?"]

stepwise_prompts = [
    "Step 1: Identify the most acidic or basic site in each molecule. Step 2: Consider how stable the molecule is after gaining or losing a proton. Step 3: Say which molecule is stronger, the first or the second, and explain why.",
    "Step 1: Find the most likely proton donor or acceptor in each molecule. Step 2: Compare their strength based on electronegativity or charge stability. Step 3: Pick the more reactive one."
]

visual_first_prompts = [
    "Examine the image only. Based on visible atom types and bonds, which molecule appears stronger in acid/base behavior? End by stating 'first' or 'second'.",
    "Look at the molecular structures and visually compare atoms and groups. Which molecule has more acidic or basic features visible in the image? Choose first or second."
]

explanation_first_prompts = [
    "The ability to stabilize charge after proton gain or loss determines acid/base strength. Analyze both molecules, then conclude which is stronger: first or second.",
    "Stronger acids form more stable conjugate bases. Based on that, explain which molecule is more acidic, and say if it’s the first or second molecule."
]

all_prompts = baseline_prompt + stepwise_prompts + visual_first_prompts + explanation_first_prompts

cot_types = (
    ["Baseline"] +
    ["Stepwise"] * len(stepwise_prompts) +
    ["Visual_first"] * len(visual_first_prompts) +
    ["Explanation_first"] * len(explanation_first_prompts)
)

# ----- 3. Define Image Pairs -----
dataset_folder = "/content"
image_pairs = [
    ("Benzoic_acid.png", "Phenol.png"),
    ("Formic_acid.png", "Acetic-acid.png"),
    ("Ammonia.png", "MeNH2.png")
]

# ----- 4. Output Setup -----
os.makedirs("outputs", exist_ok=True)
csv_file = open("outputs/Task2_deepseek_finetune-res.csv", "w", encoding="utf-8", newline="")
writer = csv.DictWriter(csv_file, fieldnames=["image1", "image2", "prompt_type", "prompt_number", "prompt_text", "generated_text"])
writer.writeheader()

# ----- 5. Inference -----
for img1, img2 in image_pairs:
    img_path1 = os.path.join(dataset_folder, img1)
    img_path2 = os.path.join(dataset_folder, img2)
    for idx, prompt_template in enumerate(all_prompts):
        prompt_type = cot_types[idx]
        conversation = [
            {
                "role": "User",
                "content": prompt_template,
                "images": [img_path1, img_path2]
            },
            {
                "role": "Assistant",
                "content": ""
            }
        ]

        images = load_pil_images(conversation)
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=images,
            force_batchify=True
        ).to(model.device)

        with torch.no_grad():
            inputs_embeds = model.prepare_inputs_embeds(**prepare_inputs)
            outputs = model.language_model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=prepare_inputs.attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=512,
                do_sample=False,
                use_cache=True
            )

        answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
        print(f"--- Finished {prompt_type} #{idx+1} for {img1} and {img2}")
        writer.writerow({
            "image1": img1,
            "image2": img2,
            "prompt_type": prompt_type,
            "prompt_number": idx + 1,
            "prompt_text": prompt_template,
            "generated_text": answer.strip()
        })

csv_file.close()
print("!!!! All pair inference completed and saved to outputs/Task2_deepseek_finetune-res.csv")
""")


In [10]:
!source /usr/local/bin/activate py38 && python task2-deepseek-finetune.py

/usr/local/envs/py38/lib/python3.8/site-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.36s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can 

In [11]:
from google.colab import files
files.download('/content/outputs/Task2_deepseek_finetune-res.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>